In [63]:
#libs
import pandas as pd 
import os 
import numpy as np
from datetime import date

In [102]:
#importando dados de contratos negociados e preço das cotações
df_ct = pd.read_pickle('cotacao_commodities.pkl')
df_tm = pd.read_pickle('negociacao_derivativos_commodities.pkl')

In [4]:
#dataframe cotações por mercadoria e bolsa negociadas
df_ct.head(2)

,data_cotacao,cod_ric,cod_ticker,cotacao_tipo,fator_conversao,bolsa_negociacao,tipo_mercadoria,mercadoria,moeda,unidade_mercadoria,valor_cotacao
0,2019-08-23,BACAINDXF,BACAINDX,FECHAMENTO,1.0,BMF,GORDO - ESALQ SÃO PAULO,BOI,REAL,ARROBA LIQUIDA,156.10
1,2019-08-23,ICFU19,AXU9,AJUSTE,1.0,BMF,ARABICO EM GRAOS- TIPO 4/5,CAFE,DOLAR DOS EUA,SACA(60KG),111.35


In [5]:
#tipo das variáveis 
df_ct.dtypes

data_cotacao          datetime64[ns]
cod_ric                       object
cod_ticker                    object
cotacao_tipo                  object
fator_conversao              float64
bolsa_negociacao              object
tipo_mercadoria               object
mercadoria                    object
moeda                         object
unidade_mercadoria            object
valor_cotacao                float64
dtype: object

In [6]:
#dataframe contratos de derivativos de commodities
df_tm.head(2)

,data_referencia,data_inicio,data_vencimento,numero_contrato,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,vendedor,comprador
0,2019-07-18,2019-07-17,2019-08-30,77902613,BORRACHA,TSR20,SGX,TONELADA METRICA,(220) USD,2,Cliente 1,Mercado
1,2019-07-18,2019-07-18,2020-01-14,77902616,MILHO,EM GRAOS A GRANEL,BMF,SACA(60KG),(790) BRL,11250,Mercado,Cliente 1


In [7]:
#tipo das variaveis
df_tm.dtypes

data_referencia         datetime64[ns]
data_inicio             datetime64[ns]
data_vencimento         datetime64[ns]
numero_contrato                 object
mercadoria                      object
tipo                            object
bolsa                           object
unidade_negociacao              object
moeda                           object
quantidade_negociada             int64
vendedor                        object
comprador                       object
dtype: object

In [103]:
#Tratando features de vendedor/comprador e criando coluna de cross das duas features
df_tm['vendedor'][df_tm['vendedor'] == 'Cliente 1'] = 'Cliente'
df_tm['comprador'][df_tm['comprador'] == 'Cliente 1'] = 'Cliente'
df_tm['comprador_vendedor'] = df_tm[['comprador', 'vendedor']].apply(lambda x: ' x '.join(x.dropna()), axis=1)

<ipython-input-103-2614472d6afc>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tm['vendedor'][df_tm['vendedor'] == 'Cliente 1'] = 'Cliente'
<ipython-input-103-2614472d6afc>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tm['comprador'][df_tm['comprador'] == 'Cliente 1'] = 'Cliente'


In [11]:
df_tm.head(2)

,data_referencia,data_inicio,data_vencimento,numero_contrato,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,vendedor,comprador,comprador_vendedor
0,2019-07-18,2019-07-17,2019-08-30,77902613,BORRACHA,TSR20,SGX,TONELADA METRICA,(220) USD,2,Cliente,Mercado,Mercado x Cliente
1,2019-07-18,2019-07-18,2020-01-14,77902616,MILHO,EM GRAOS A GRANEL,BMF,SACA(60KG),(790) BRL,11250,Mercado,Cliente,Cliente x Mercado


In [13]:
df_ct.head(2)

,data_cotacao,cod_ric,cod_ticker,cotacao_tipo,fator_conversao,bolsa_negociacao,tipo_mercadoria,mercadoria,moeda,unidade_mercadoria,valor_cotacao
0,2019-08-23,BACAINDXF,BACAINDX,FECHAMENTO,1.0,BMF,GORDO - ESALQ SÃO PAULO,BOI,REAL,ARROBA LIQUIDA,156.10
1,2019-08-23,ICFU19,AXU9,AJUSTE,1.0,BMF,ARABICO EM GRAOS- TIPO 4/5,CAFE,DOLAR DOS EUA,SACA(60KG),111.35


In [104]:
#tratamento das chaves de união das bases de cmm e ctc
lista_tm = ['mercadoria','tipo','bolsa','unidade_negociacao','moeda']
for l in lista_tm:
    df_tm[l] = df_tm[l].str.strip()

lista_ct = ['bolsa_negociacao','tipo_mercadoria','mercadoria','moeda','unidade_mercadoria']    
for l in lista_ct:
    df_ct[l] = df_ct[l].str.strip()


In [105]:
#merge dos datasets
df_merge = df_tm.merge(df_ct, how='inner', 
                      left_on=(['data_referencia','mercadoria','tipo','bolsa','unidade_negociacao']),
                      right_on=(['data_cotacao','mercadoria','tipo_mercadoria','bolsa_negociacao','unidade_mercadoria']))

In [18]:
#imprimindo as colunas do dataframe resultado do merge 
df_merge.columns

Index(['data_referencia', 'data_inicio', 'data_vencimento', 'numero_contrato',
       'mercadoria', 'tipo', 'bolsa', 'unidade_negociacao', 'moeda_x',
       'quantidade_negociada', 'vendedor', 'comprador', 'comprador_vendedor',
       'data_cotacao', 'cod_ric', 'cod_ticker', 'cotacao_tipo',
       'fator_conversao', 'bolsa_negociacao', 'tipo_mercadoria', 'moeda_y',
       'unidade_mercadoria', 'valor_cotacao'],
      dtype='object')

In [106]:
#deletando as colunas repetidas
df = df_merge.drop(columns=(['data_inicio','numero_contrato','data_vencimento','vendedor','comprador','data_cotacao',
                               'cotacao_tipo','bolsa_negociacao','tipo_mercadoria','moeda_y','unidade_mercadoria']))
#renomeando a coluna moeda
df.rename(columns={'moeda_x':'moeda'}, inplace=True)

In [9]:
df.tail()

,data_referencia,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,fator_conversao,valor_cotacao
242573,2022-05-18,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOV2,COV2,1.0,102.33
242574,2022-05-18,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOV3,COV3,1.0,88.99
242575,2022-05-18,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOX2,COX2,1.0,100.46
242576,2022-05-18,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOX3,COX3,1.0,88.32
242577,2022-05-18,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOZ2,COZ2,1.0,98.76


- Para obter o valor real dos contratos de derivativos na moeda brasileira (BRL) e fazer estudos da variação de câmbio, torna-se necessário enriquecer a nossa base de dados com cotação diária do dolar. 
- Além disso, para correção real do valor negociado com contratos passados, será adicionada a variável de ipca mensal, índice que mensura a inflação nacional. 

- Os dados são obtidos a partir de uma biblioteca denominada bcb.
    documentação: https://pypi.org/project/python-bcb/

In [107]:
#importar dados de usd
#Conversor de Moedas: O Conversor de Moedas realiza o Web Scrapping do sistema de mesmo nome do Banco Central. Permite que seja importado as cotações diárias de diversas moedas diante do real. Para saber as moedas que são possíveis de importar, e consequentemente seus respectivos símbolos, utiliza-se a função currency.get_currency_list().
from bcb import currency
from bcb import sgs

lista_moeda = currency.get_currency_list()

In [109]:
#criando um dataframe com dados diários do dolar
usd = currency.get(['USD'], start='2018-12-28', end='2022-07-14')
usd.head(5)

,USD
Date,
2018-12-28,3.8748
2018-12-31,3.8748
2019-01-02,3.8595
2019-01-03,3.7683
2019-01-04,3.7627


In [6]:
usd.dtypes

USD    float64
dtype: object

In [110]:
from bcb import sgs
ipca = sgs.get({'IPCA':433}, start='2018-01-01', end='2022-07-01')
selic = sgs.get({'SELIC':432}, start='2019-01-01', end='2022-07-01')
igpm = sgs.get({'igp-m':189}, start='2019-01-01', end='2022-07-01')

In [111]:
ind = pd.DataFrame()

ind['igpm']=igpm['igp-m']
ind['igpm_acumulado']=igpm['igp-m'].cumsum()
ind['selic']=selic['SELIC']
ind['ipca']=ipca['IPCA']
ind['ipca_acumulado']=ipca['IPCA'].cumsum()
#taxas['ipca_acumulado']=12
#taxas['ipca_acumulado'][1:]=taxas['ipca_acumulado'][0]+taxas['ipca'
ind['juros_real']=selic['SELIC']-ipca['IPCA']
ind['usd']=usd['USD']
ind.head(2)

,igpm,igpm_acumulado,selic,ipca,ipca_acumulado,juros_real,usd
Date,,,,,,,
2019-01-01,0.01,0.01,6.5,0.32,4.01,6.18,NaN
2019-02-01,0.88,0.89,6.5,0.43,4.44,6.07,3.6694


In [112]:
#criando dataframe com dados mensais de ipca 
ipca = sgs.get({'IPCA':433}, start='2019-01-01', end='2022-07-01')
#adicionando uma coluna do acumulado do ipca
ipca['ipca_acumulado'] = ipca['IPCA'].cumsum()
ipca.head(2)

,IPCA,ipca_acumulado
Date,,
2019-01-01,0.32,0.32
2019-02-01,0.43,0.75


In [8]:
ipca.dtypes

IPCA              float64
ipca_acumulado    float64
dtype: object

In [113]:
#adicionando data_referencia como index do dataframe principal para adicionar o dolar por dia 
df.set_index('data_referencia', inplace=True)

In [106]:
df.head(2)

,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,fator_conversao,valor_cotacao
data_referencia,,,,,,,,,,,
2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOH0,HOH0,1.0,1.9152
2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOX0,HOX0,1.0,1.8636


In [114]:
#adicionando coluna de usd no dataframe principal
df['usd'] = usd['USD']

In [115]:
#criando coluna de volume (notional) em usd e brl
df['notional_usd'] = df['quantidade_negociada']*df['valor_cotacao']
df['notional_brl'] = np.where(df['moeda']=='(220) USD', df['notional_usd']*df['usd'], df['notional_usd'])

In [23]:
df.head(2)

,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,fator_conversao,valor_cotacao,usd,notional_usd,notional_brl
data_referencia,,,,,,,,,,,,,,
2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOH0,HOH0,1.0,1.9152,3.7596,6595948.8,2.479813e+07
2019-07-24,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOX0,HOX0,1.0,1.8636,3.7596,6418238.4,2.413001e+07


In [132]:
#resetando o index para criar coluna anomes
ind.reset_index(inplace=True)
df.reset_index(inplace=True)

In [118]:
#criando coluna de ano e mes 
ind['mes'] = ind.Date.dt.month.astype(str)
ind['ano'] = ind.Date.dt.year.astype(str)
ind['anomes'] = ind[['ano', 'mes']].apply(lambda x: ''.join(x.dropna()), axis=1)
# dataframe principal
df['mes'] = df.data_referencia.dt.month.astype(str)
df['ano'] = df.data_referencia.dt.year.astype(str)
df['anomes'] = df[['ano', 'mes']].apply(lambda x: ''.join(x.dropna()), axis=1)

In [133]:
#setando index para juntar os dois dataframes
df.set_index('data_referencia', inplace=True)
ind.set_index('Date', inplace=True)

In [134]:
ind.tail(2)

,level_0,index,anomes,igpm,igpm_acumulado,selic,ipca,ipca_acumulado,juros_real,usd,mes,ano
Date,,,,,,,,,,,,
2022-06-01,41,41,20226,0.59,52.71,12.75,0.67,27.36,12.08,4.7765,6,2022
2022-07-01,42,42,20227,0.21,52.92,13.25,-0.68,26.68,13.93,5.3142,7,2022


In [135]:
#adicionando dados do ipca no dataframe principal
df['ipca'] = ind['ipca']
df['ipca_acumulado'] = ind['ipca_acumulado']
df['selic']=ind['selic']
df['igpm']=ind['igpm']
df['igpm_acumulado']=ind['igpm_acumulado']
df['juros_real']=ind['juros_real']
df['contratos']=1
#corrigindo ipca
df['notional_brl_ipca']=(df['notional_brl']*df['ipca_acumulado'])/df['ipca'][0]
df['notional_usd_ipca']=(df['notional_usd']*df['ipca_acumulado'])/df['ipca'][0]


In [138]:
df.tail(2)

,anomes,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,...,ano,ipca,ipca_acumulado,selic,igpm,igpm_acumulado,juros_real,contratos,notional_brl_ipca,notional_usd_ipca
data_referencia,,,,,,,,,,,,,,,,,,,,,
2022-05-18,20225,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOX3,COX3,...,2022,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN
2022-05-18,20225,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOZ2,COZ2,...,2022,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN


In [84]:
df.tail()

,index,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,...,ipca,ipca_acumulado,selic,igpm,igpm_acumulado,juros_real,contratos,notional_brl_ipca,notional_usd_ipca,notionalbrl_norm
data_referencia,,,,,,,,,,,,,,,,,,,,,
2022-05-18,242573,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOV2,COV2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0.110672
2022-05-18,242574,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOV3,COV3,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0.061308
2022-05-18,242575,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOX2,COX2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0.103752
2022-05-18,242576,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOX3,COX3,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0.058829
2022-05-18,242577,PETROLEO,PRIMEIRO FUTURO OLEO CRU-BRENT,ICE (NYBOT),BARRIL,(220) USD,31000,Mercado x Cliente,LCOZ2,COZ2,...,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0.097461


In [81]:
df.reset_index(inplace=True)
df.set_index('data_referencia', inplace=True)

In [83]:
#salvando dataframe principal
df.to_pickle('mastertable.pkl')

In [82]:
df['notionalbrl_norm'] = (df['notional_brl'] - df['notional_brl'].mean())/df['notional_brl'].std()

In [35]:
df.head(2)

,anomes,mercadoria,tipo,bolsa,unidade_negociacao,moeda,quantidade_negociada,comprador_vendedor,cod_ric,cod_ticker,...,notional_brl,mes,ano,ipca,ipca_acumulado,selic,igpm,igpm_acumulado,juros_real,notionalbrl_norm
data_referencia,,,,,,,,,,,,,,,,,,,,,
2019-07-24,20197,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOH0,HOH0,...,2.479813e+07,7,2019,0.19,6.09,6.5,0.4,4.72,6.31,0.328900
2019-07-24,20197,OLEO,NY HARBOR ULSD FUTURES (HO),NYMEX,GALAO,(220) USD,3444000,Mercado x Cliente,HOX0,HOX0,...,2.413001e+07,7,2019,0.19,6.09,6.5,0.4,4.72,6.31,0.312819
